In [55]:
import tensorflow as tf
import numpy as np
import numpy.random as rnd
from sklearn.datasets import make_moons
from sklearn.preprocessing import StandardScaler

In [56]:
# TensorBoard stuff
from datetime import datetime

now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "tf_logs"
logdir = "{}/run-{}/".format(root_logdir, now)

In [57]:
tf.reset_default_graph()

# Define the learning rate and number of epochs
n_epochs = 5000
learning_rate = 0.01

# Load and scale dataset
moons = make_moons()
moons_plus_bias = np.c_[np.ones((moons[0].shape[0], 1)), moons[0]]
scaled_moons_plus_bias = StandardScaler().fit_transform(moons_plus_bias)

# Define batch size and number of batches
m, n = moons[0].shape
batch_size = 100
n_batches = int(np.ceil(m / batch_size))

In [58]:
# Mini-batch Gradient Descent
def fetch_batch(epoch, batch_index, batch_size):
    rnd.seed(epoch * n_batches + batch_index)
    indices = rnd.randint(m, size=batch_size)
    X_batch = scaled_moons_plus_bias[indices]
    y_batch = moons[1].reshape(-1, 1)[indices]
    return X_batch, y_batch

In [59]:
def logistic_regression():
    with tf.name_scope("inputs") as scope:
        X = tf.placeholder(tf.float32, shape=(None, n + 1), name="X")
        y = tf.placeholder(tf.float32, shape=(None, 1), name="y")
    
    theta = tf.Variable(tf.random_uniform([moons[0].shape[1] + 1, 1], -1.0, 1.0), name="theta")
    y_pred = tf.nn.sigmoid(tf.matmul(X, theta, name="predictions"))
    
    with tf.name_scope("loss") as scope:
        error = (-y * tf.log(y_pred) - (1-y) * tf.log(1 - y_pred)) 
        cost = tf.reduce_mean(error, name = "cost")

    optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
    training_op = optimizer.minimize(cost)
    init = tf.global_variables_initializer()
    saver = tf.train.Saver()
    
    # Summary for plotting
    cost_summary = tf.summary.scalar('MSE', cost)
    file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())
    
    # Train linear regression model using mini-batch gradient descent
    with tf.Session() as sess:
        sess.run(init)
        for epoch in range(n_epochs):
            for batch_index in range(n_batches):
                X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
                if batch_index % 10 == 0:
                    summary_str = cost_summary.eval(feed_dict={X: X_batch, y: y_batch})
                    step = epoch * n_batches + batch_index
                    file_writer.add_summary(summary_str, step)
                sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        best_theta = theta.eval()
        # Save the best model
        save_path = saver.save(sess, "logistic_regression.ckpt")
        file_writer.close()
    
    return best_theta

In [60]:
logistic_regression()

array([[  1.75642967e-03],
       [  1.00682271e+00],
       [ -2.51997995e+00]], dtype=float32)